In [1]:
import os

In [2]:
%pwd

"c:\\Users\\pc\\OneDrive - Institut National de Statistique et d'Economie Appliquee\\Documents\\GitHub\\Text-Summarization-NLP-Project\\research"

In [3]:
os.chdir("../")

In [4]:
%pwd

"c:\\Users\\pc\\OneDrive - Institut National de Statistique et d'Economie Appliquee\\Documents\\GitHub\\Text-Summarization-NLP-Project"

In [ ]:
from pathlib import Path
from dataclasses import dataclass   

@dataclass(frozen=True)   
class ModelTrainerConfig : 
    root_dir : Path
    data_path: Path
    model_name : Path
    num_train_epochs : int
    warmup_steps : int
    per_device_train_batch_size : int
    per_device_eval_batch_size : int
    weight_decay : float
    logging_steps : int
    evaluation_strategy : str
    eval_steps : int
    save_steps : float
    gradient_accumulation_steps : int